<a href="https://colab.research.google.com/github/chinmayj23/ML/blob/master/ML_lab9_J079.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import re
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

np.random.seed(27)

In [3]:
data=pd.read_csv("/content/drive/My Drive/Surveillance.csv")

In [4]:
data.head()

,A01,A02,A03,A04,A05,A06,A07,Categories
0,+,+,+,+,+,-,-,PUS
1,+,+,-,+,+,-,-,PUS
2,+,+,+,+,-,+,-,PUS
3,+,+,-,+,-,+,-,PUS
4,+,-,-,-,-,-,+,PUS


In [5]:
df=data.drop(columns=['Categories'],axis=1)
y=data['Categories']

In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [7]:
df=df.apply(le.fit_transform)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [9]:
seed = 8
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=test_size, random_state=seed)

In [10]:
from sklearn import linear_model
logistic = linear_model.LogisticRegression(solver='liblinear')
sgd = linear_model.SGDClassifier()

In [11]:
models = [logistic, sgd]

In [25]:
penalty = ['l1', 'l2']
C = [0.0001, 0.001, 0.01]
class_weight = [{1:0.5, 0:0.5}, {1:0.4, 0:0.6}]
solver = ['liblinear', 'saga']

param_grid = dict(penalty=penalty,
                  C=C,
                  class_weight=class_weight,
                  solver=solver)

grid = GridSearchCV(estimator=logistic, param_grid=param_grid, scoring='accuracy', verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

print('Best Score: ', grid_result.best_score_)
print('Best Params: ', grid_result.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Best Score:  0.4666666666666666
Best Params:  {'C': 0.01, 'class_weight': {1: 0.5, 0: 0.5}, 'penalty': 'l2', 'solver': 'saga'}


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    1.5s finished
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [24]:
grid_result.best_params_

{'C': 0.0001,
 'class_weight': {0: 0.5, 1: 0.5},
 'penalty': 'l1',
 'solver': 'saga'}

In [20]:
def get_cv_scores(model):
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
    print('CV Mean: ', np.mean(scores))
    print('STD: ', np.std(scores))
    print('\n')

In [21]:
logistic = linear_model.LogisticRegression(C=0.0001, class_weight={1:0.5, 0:0.5}, penalty='l2', solver='liblinear')
get_cv_scores(logistic)

CV Mean:  0.5555555555555555
STD:  0.0785674201318386




/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  % (min_groups, self.n_splits)), UserWarning)


In [22]:

from sklearn.metrics import accuracy_score

logistic.fit(X_train, y_train)

y_train_pred = logistic.predict(X_train)
accuracy_train = accuracy_score(y_train, y_train_pred)
print("Accuracy: %.2f%%" % (accuracy_train))


y_test_pred = logistic.predict(X_test)
accuracy_test = accuracy_score(y_test, y_test_pred)
print("Accuracy: %.2f%%" % (accuracy_test))

Accuracy: 0.55%
Accuracy: 0.67%


In [26]:
from sklearn import ensemble

In [28]:
RF=ensemble.RandomForestClassifier()

In [29]:
RF.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [35]:
ypred=RF.predict(X_test)
ypred_train=RF.predict(X_train)

In [36]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train,ypred_train))
print(accuracy_score(y_test,ypred))

1.0
0.6666666666666666
